# Random Forest
Predict CNN attention per nucleus.   
Accuracy was low in notebook 03.    
Here, don't remove the "bad cols".   

In [1]:
MAX_DEPTH = 8
N_ESTIMATORS = 500
CHOOSE_EVERY = 10
SAVE_MEM = True

In [2]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import csv
import numpy as np
import pandas as pd
import matplotlib as plt
import random
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
# The model
from sklearn.ensemble import RandomForestClassifier

2022-08-11 17:30:59.057712
Python 3.8.10
sklearn 1.0.2


In [3]:
# Patch data.
# This is the nucleus-level csv file: one row per nucleus from CellProfiler.
FILENAME='Process100_Nucleus.csv'
# This directory contains one Nucleus.csv file per patient.
BASE_PATH='D:\\Adjeroh\\Glioma\\August_Run\\CellProfilerPerPatient\\'   # Windows
BASE_PATH='/home/jrm/Adjeroh/Glioma/August_Run/CellProfilerPerPatient/'  # Alien

# Patient data.
# Each patch filename indicates patient/case [0:19] and WSI [0:23].
# For example: TCGA-06-0129-01Z-00-DX1_5400_5100.png
LEN_PATIENT_ID = 19
FOLDS_META_FILE = 'D:\\Adjeroh\\Glioma\\August_Run\\TrainValidSplits\\aug_train_valid_splits.csv'  # Windows
FOLDS_META_FILE = '/home/jrm/Adjeroh/Glioma/August_Run/TrainValidSplits/aug_train_valid_splits.csv'  # Alien
# Cross validation
NUM_FOLDS = 5

# These bin thresholds yield nucleus temperature bins of about equal weight.
# See AT_PerPatient 01 notebook.
BIN_MAX = [-0.99,0.40,0.65,0.75,0.85,1.00]
BIN_LABEL = [1,2,3,4,5]

In [4]:
# Expect all temperatures to fall into bins 1 thru 5.
# Bins 0 and 6 should never happen.
every_temp = [0.35,0.50,0.60,0.70,0.90,1.10]
binned = np.digitize(every_temp,BIN_MAX,right=True)
binned

array([1, 2, 2, 3, 5, 6])

## Data loading functions

In [5]:
METADATA = pd.read_csv(FOLDS_META_FILE)

In [6]:
def get_patient_attention(fold,is_train,patient):
    cls = 'mcls'
    filename = f"Attention_{cls}_fold{fold}.csv"
    filepath = BASE_PATH + patient + '/' + filename
    df = pd.read_csv(filepath)
    return df

In [7]:
def get_metadata_one_fold(fold,is_train):
    df = METADATA.loc[METADATA['Fold']==fold]
    if is_train:
        df = df.loc[df['Split']=='Train']
    else:
        df = df.loc[df['Split']=='Valid']
    patients = df['Case'].tolist()
    labels = df['Class'].tolist()
    return patients,labels

In [8]:
# What happens if we don't drop the "bad cols"?
def drop_columns(df):
    bad_cols = ['ImageNumber','ObjectNumber','Number_Object_Number']
    #            'AreaShape_Orientation','Children_Cells_Count']
    df = df.drop(columns=bad_cols)
    bad_cols = [c for c in df.columns 
                if c.startswith('Location_') 
                or c.startswith('AreaShape_BoundingBoxM')
                or c.startswith('AreaShape_Center')
                or c.startswith('Neighbors_')]
    # df = df.drop(columns=bad_cols)
    return df

In [9]:
def load_one_patient_attention(fold,multiclass,patient):
    filepath=BASE_PATH+patient+'/'+FILENAME
    X = pd.read_csv(filepath)
    cls='bcls'
    if multiclass:
        cls='mcls'
    att_filename = f"Attention_{cls}_fold{fold}.csv"
    att_path = BASE_PATH+patient+'/'+att_filename
    att_df = pd.read_csv(att_path)
    thermometer = {}
    for ndx,row in att_df.iterrows():
        key = (row['ImageNumber'],row['ObjectNumber'])
        value = row['AvgTemp']
        thermometer[key]=value
    every_temp = np.zeros(len(X),dtype=float)
    i = 0
    for ndx,row in X.iterrows():
        key = (row['ImageNumber'],row['ObjectNumber'])
        value = thermometer[key]
        every_temp[i] = value
        i += 1
    binned = np.digitize(every_temp,BIN_MAX,right=True)
    for i in range(len(binned)):
        bin_num = binned[i]
        if not (bin_num>=1 and bin_num<=5):
            print(i,binned[i],every_temp[i],patient)
            raise Exception ('Bad bin!')
    Y = list(binned) 
    return X,Y  

In [10]:
def load_all_patients_data(fold,patients,save_mem=False):
    X = None  # dataframe with 600 feature columns, one row per nucleus
    y = None  # list of labels = attention for each row
    z = None  # list of patient name for each row
    is_multiclass = True
    count=0
    for patient in patients:
        print('.',end='')
        count += 1
        directory = patient+'/'
        Xall,yall = load_one_patient_attention(fold,is_multiclass,patient)
        if (save_mem):
            Xi = Xall.iloc[0::CHOOSE_EVERY] # e.g. choose every tenth patch
            yi = yall[0::CHOOSE_EVERY].copy()  # e.g. choose every tenth patch
        else:
            Xi = Xall
            yi = yall
        zi = [patient]*len(Xi)  # same ID for all patches from one patient
        if X is None:
            X = Xi
            y = yi
            z = zi
        else:
            X = pd.concat( (X, Xi) )
            y = np.concatenate( (y, yi) )
            z = np.concatenate( (z, zi) )
    print()
    X = drop_columns(X)
    return X,y,z   # nucleus data, nucleus attention bin, patient name

In [11]:
# This is slow, probably due to concatenating dataframes.
# Consider a rewrite that concatenates csv before constructing a dataframe.
def load_train_valid_set(fold):
    is_train = True # training set
    patients,cancers = get_metadata_one_fold(fold,is_train) 
    X_train,y_train,z_train = load_all_patients_data(fold,patients,SAVE_MEM) 
    print('Train lengths X,y,z:',X_train.shape,len(y_train),len(z_train))
    is_train = False # validation set
    patients,cancers = get_metadata_one_fold(fold,is_train) 
    X_valid,y_valid,z_valid = load_all_patients_data(fold,patients,SAVE_MEM) # reduces validation size
    print('Valid lengths X,y,z:',X_valid.shape,len(y_valid),len(z_valid))
    return X_train,y_train,z_train,X_valid,y_valid,z_valid

## Aggregation functions

In [12]:
# Each patch votes on the class for its WSI.
# Later, weight each patch label by the confidence i.e. score
def aggregate_accuracy(y_pred,y_valid,z_valid):
    L = len(y_pred)
    if L != len(y_valid) or L != len(z_valid):
        raise Exception('Lengths do not match')
    correct = {}
    incorrect = {}
    patients = np.unique(z_valid)
    for patient in patients:
        correct[patient]=0  # accumulate correct votes on patches
        incorrect[patient]=0  # accumulate incorrect votes
    for i in range(L):
        patient = z_valid[i]
        label =   y_valid[i]
        pred =    y_pred[i]
        if pred == label:
            correct[patient] += 1
        else:
            incorrect[patient] += 1
    numerator = 0
    denominator = 0
    for patient in patients:
        denominator += 1
        if correct[patient]>incorrect[patient]:
            numerator += 1
    accuracy = float(0)
    if denominator>0:
        accuracy = 100.0*numerator/denominator 
    return accuracy

## Main loop: Load, Classify, Report

In [13]:
def important_features(model):
    # Prereqs: fit().
    names = model.feature_names_in_
    importances = model.feature_importances_
    pairs = np.column_stack( (names,importances) )
    top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
    # There must be a way to do this witout a loop!
    top_list = []
    for i in top_array:
         top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
    top_df = pd.DataFrame(top_list)
    return top_df

In [14]:
patch_accuracies = []   # summary statistics
patient_accuracies = [] # summary statistics

for fold in range(NUM_FOLDS):
    print()
    print(datetime.datetime.now(),'Fold',fold,'Loading...')
    X_train,y_train,z_train,X_valid,y_valid,z_valid = load_train_valid_set(fold)    

    # The shuffle is not strictly necessary with RF.
    # CNN models are sensitive to train set order but RF models are not. 
    # print(datetime.datetime.now(),'Shuffle...')
    # X_train,y_train = sklearn.shuffle(X_train,y_train)

    print(datetime.datetime.now(),'Train...')
    # min_samples_leaf=1 (default) led to overfitting
    rfc = RandomForestClassifier(max_depth=MAX_DEPTH,n_estimators=N_ESTIMATORS)
    rfc.fit(X_train,y_train)  # slow
    
    print(datetime.datetime.now(),'Ranked feature imporances...')
    top = important_features(rfc)
    #pd.set_option('display.max_rows', None)
    print(top.loc[:10])
    
    print(datetime.datetime.now(),'Evaluate...')
    y_pred = rfc.predict(X_train)
    matches = np.count_nonzero(y_train==y_pred)
    accuracy_t = 100.0 * matches / len(y_pred)
    print('Fold',fold,'Nucleus-level Training Accuracy:',accuracy_t)
    
    print(datetime.datetime.now(),'Validate...')
    y_pred = rfc.predict(X_valid)
    matches = np.count_nonzero(y_valid==y_pred)
    accuracy_v = 100.0 * matches / len(y_pred)
    print('Fold',fold,'Nucleus-level Validation Accuracy:',accuracy_v)
    patch_accuracies.append(accuracy_v)
    
    accuracy_p = aggregate_accuracy(y_pred,y_valid,z_valid)
    patient_accuracies.append(accuracy_p)
    print('Fold',fold,'Patient-level Validation Accuracy:',accuracy_p)
    
    # This shouldn't be necessary but it seems to reduce memory footprint.
    X_train=None
    X_valid=None
    y_train=None
    y_valid=None
    z_train=None
    z_valid=None
    y_pred = None
    matches = None


2022-08-11 17:30:59.791300 Fold 0 Loading...
.....................................................................................................
Train lengths X,y,z: (148324, 647) 148324 148324
..........................
Valid lengths X,y,z: (36559, 647) 36559 36559
2022-08-11 17:34:42.670083 Train...
2022-08-11 17:46:34.498292 Ranked feature imporances...
           0                                           1
0   0.092723                           Location_Center_X
1   0.083816                          AreaShape_Center_X
2   0.080191  Location_CenterMassIntensity_X_Hematoxylin
3   0.077906         Location_MaxIntensity_X_Hematoxylin
4   0.077256              AreaShape_BoundingBoxMinimum_X
5   0.070092              AreaShape_BoundingBoxMaximum_X
6   0.061202              AreaShape_BoundingBoxMinimum_Y
7   0.059463  Location_CenterMassIntensity_Y_Hematoxylin
8   0.056002                          AreaShape_Center_Y
9   0.055664                           Location_Center_Y
10  0.05456

In [15]:
print(datetime.datetime.now())
if SAVE_MEM:
    print('Sampling every',CHOOSE_EVERY,'th record.')
print('Cross validation nucleus-level accuracy:',patch_accuracies)
print('mean:',np.mean(patch_accuracies),'pop std:',np.std(patch_accuracies,ddof=1))
print('Cross validation patient-level accuracy:',patient_accuracies)
print('mean:',np.mean(patient_accuracies),'pop std:',np.std(patient_accuracies,ddof=1))


2022-08-11 18:48:16.303043
Sampling every 10 th record.
Cross validation nucleus-level accuracy: [52.43578872507454, 44.2775608290778, 42.57037183432621, 47.64832369241952, 48.854331004432424]
mean: 47.1572752170661 pop std: 3.883177481257632
Cross validation patient-level accuracy: [53.84615384615385, 15.384615384615385, 4.0, 32.0, 36.0]
mean: 28.246153846153845 pop std: 19.254016717643974
